In [2]:
# Paste your API key for IBM Watson Visual Recognition below: OSKAR
my_apikey = 'B5b3G2VOKRLtUDxgk0GenbY_qv59-OdotwhdWK29x8gJ'
import time

In [3]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2020-04-13', 
                             authenticator=authenticator)

import json
with open('Motor.zip', 'rb') as Motor, \
     open('PP.zip', 'rb') as PowerPlant, \
     open('SS.zip', 'rb') as Substation, \
     open('Transformer.zip', 'rb') as Transformer, \
     open('Transmission Lines.zip', 'rb') as TransmissionLine:
        response = visrec.create_classifier(name="Device Classifier",
                                        positive_examples={'Motor': Motor, \
                                                           'Power plant': PowerPlant, \
                                                           'Substation': Substation, \
                                                           'Transformer': Transformer, \
                                                           'Transmission line': TransmissionLine})
print(json.dumps(response.get_result(), indent=2))

In [4]:
#lets grab the classifier id
classifier_id = 'DeviceClassifier_1744264373'
classifier_id

'DeviceClassifier_1744264373'

Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

print(time.ctime())
while(True):
    Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
    if Status!='training': 
        print(time.ctime())
        break

In [5]:
visrec.list_classifiers(verbose=True).get_result()

{'classifiers': [{'classifier_id': 'DeviceClassifier_1744264373',
   'name': 'Device Classifier',
   'status': 'ready',
   'owner': '706f8941-1721-4c87-8296-8c489ffe8cca',
   'created': '2020-04-13T07:23:27.025Z',
   'updated': '2020-04-13T07:23:27.025Z',
   'classes': [{'class': 'PP'},
    {'class': 'Transformer'},
    {'class': 'Transmission Lines'},
    {'class': 'SS'},
    {'class': 'Motor'}],
   'core_ml_enabled': True}]}

import json
visual_recognition = VisualRecognitionV3(
    version='2018-03-19',
    authenticator=authenticator
)

with open('./Motor 1.jpg', 'rb') as images_file:
    classes = visual_recognition.classify(
        images_file=images_file,
        threshold='0.6',
        owners=["me"]).get_result()
    print(json.dumps(classes, indent=2))

In [14]:
from pandas.io.json import json_normalize

def getdf_visrec(images_file, apikey = my_apikey):
    
    json_result = visrec.classify(images_file = images_file,
                              threshold='0.6',
                              classifier_ids=classifier_id).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return json_classes

In [15]:
getdf_visrec(images_file = open('./Motor 1.jpg', 'rb'))[0]['class']

'Motor'